In [1]:
import pandas as pd

/tmp/ipykernel_2117/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('wustl_iiot_2021.csv')

In [3]:
df

,StartTime,LastTime,SrcAddr,DstAddr,Mean,Sport,Dport,SrcPkts,DstPkts,TotPkts,...,SAppBytes,DAppBytes,TotAppByte,SynAck,RunTime,sTos,SrcJitAct,DstJitAct,Traffic,Target
0,2019-08-19 12:23:28,2019-08-19 12:23:28,192.168.0.20,192.168.0.2,0,59034,502,10,8,18,...,24,20,44,0.001176,0.053037,0,0.000000,0.0,normal,0
1,2019-08-19 15:13:24,2019-08-19 15:13:24,192.168.0.20,192.168.0.2,0,55841,502,10,8,18,...,24,20,44,0.001308,0.052681,0,0.000000,0.0,normal,0
2,2019-08-19 13:41:31,2019-08-19 13:41:31,192.168.0.20,192.168.0.2,0,63774,502,10,8,18,...,24,20,44,0.000690,0.051793,0,0.000000,0.0,normal,0
3,2019-08-19 12:43:19,2019-08-19 12:43:20,209.240.235.92,192.168.0.2,0,61771,80,4,0,4,...,0,0,0,0.000000,0.889555,0,419.338813,0.0,DoS,1
4,2019-08-19 14:49:44,2019-08-19 14:49:48,192.168.0.20,192.168.0.1,3,0,0,14,0,14,...,476,0,476,0.000000,3.500055,0,525.146562,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194459,2019-08-19 16:41:47,2019-08-19 16:41:47,192.168.0.20,192.168.0.2,0,57730,502,10,8,18,...,24,20,44,0.001238,0.053972,0,0.000000,0.0,normal,0
1194460,2019-08-19 14:39:47,2019-08-19 14:39:47,192.168.0.20,192.168.0.2,0,59930,502,6,6,12,...,24,22,46,0.000646,0.032614,0,0.000000,0.0,normal,0
1194461,2019-08-19 14:06:18,2019-08-19 14:06:18,192.168.0.20,192.168.0.2,0,64384,502,10,8,18,...,24,20,44,0.000668,0.052755,0,0.000000,0.0,normal,0
1194462,2019-08-19 13:20:52,2019-08-19 13:20:52,192.168.0.20,192.168.0.2,0,53833,502,10,8,18,...,24,20,44,0.000676,0.051783,0,0.000000,0.0,normal,0


## Preprocess

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sklearn.preprocessing import MinMaxScaler

for col in df.columns:
    
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
df = df.fillna(0)
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Target','Traffic'])
scaler = MinMaxScaler()

numeric_columns = X.select_dtypes(include=[np.number]).columns
X[numeric_columns] = scaler.fit_transform(df[numeric_columns])

X[numeric_columns] =X[numeric_columns].astype(np.float64)

# Define target variable y
y = df['Target']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the training set into training and validation sets (further split to get 20% of the original data as validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.to_numpy()).to(device)
y_train_tensor = torch.LongTensor(y_train.values).to(device)
X_val_tensor = torch.FloatTensor(X_val.to_numpy()).to(device)
y_val_tensor = torch.LongTensor(y_val.values).to(device)
X_test_tensor = torch.FloatTensor(X_test.to_numpy()).to(device)
y_test_tensor = torch.LongTensor(y_test.values).to(device)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create data loaders
batch_size = 64  # You can adjust this
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

class DNN(nn.Module):
    def __init__(self, input_size):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 2) 

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

# Determine the input size for the first layer of the network
input_size = X_train_tensor.shape[1]

# Instantiate the model
dnn_model = DNN(input_size)

dnn_model.to(device)

DNN(
  (layer1): Linear(in_features=47, out_features=128, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=2, bias=True)
)

In [6]:
print(X_train_tensor.device)
print(y_train_tensor.device)

cuda:0
cuda:0


In [7]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(dnn_model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # You can adjust this
for epoch in range(num_epochs):
    dnn_model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = dnn_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Optionally, you can add a validation step here

print('Training completed')


Epoch [1/10], Loss: 0.0000
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0005
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Training completed


In [8]:
# Ensure the model is in evaluation mode
dnn_model.eval()

# No gradient is needed for evaluation
with torch.no_grad():
    y_pred = []
    y_true = []

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = dnn_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Convert lists to numpy arrays for metric calculation
y_pred = np.array(y_pred)
y_true = np.array(y_true)

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Calculating TPR, FPR, FNR, TNR
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
fnr = fn / (tp + fn)
tnr = tn / (tn + fp)

# Other metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)  # Same as TPR
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

# Print the metrics
print(f"Accuracy of DNN: {accuracy:.4f}")
print(f"Precision of DNN: {precision:.4f}")
print(f"Recall (TPR) of DNN: {recall:.4f}")
print(f"F1 Score of DNN: {f1:.4f}")
print(f"AUC of DNN: {auc:.4f}")
print(f"FPR of DNN: {fpr:.4f}")
print(f"FNR of DNN: {fnr:.4f}")
print(f"TNR of DNN: {tnr:.4f}")

Accuracy of DNN: 0.9999
Precision of DNN: 0.9990
Recall (TPR) of DNN: 0.9992
F1 Score of DNN: 0.9991
AUC of DNN: 0.9996
FPR of DNN: 0.0001
FNR of DNN: 0.0008
TNR of DNN: 0.9999
